In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from geopack import geopack

Load IGRF coefficients ...


In [3]:
#open all .txt files in a directory in pandas

def open_files(directory):
    files = os.listdir(directory)
    dataframes = []
    for file in files:
        if file.endswith('.txta'):

            data = pd.read_csv(directory+file, sep='\s+', header=None)
            dataframes.append(data)
        
            df = pd.concat(dataframes)

    df['dt'] = pd.to_datetime((df[0].astype(str) + df[1].astype(str) + df[2].astype(str)), format='%Y%m%d%H%M')
    
    #drop first 3 columns and move last column to first
    df = df.drop([0,1,2,7,8,9,10], axis=1)
    cols = list(df.columns)
    cols = [cols[-1]] + cols[:-1]
    df = df[cols]

    df = df.sort_values(by='dt')  
    df = df.reset_index(drop=True)

    df_labs = ['dt', 'np', 'vsw', 'By', 'Bz']
    df.columns = df_labs
    
    #apply constants to columns
    df['np'] = df['np'] * 10 #in cm^-3
    df['vsw'] = (df['vsw'] * 5.32e4) / 1000 #in km/s
    df['By'] = df['By'] * 10 #in nT
    df['Bz'] = df['Bz'] * 10    #in nT

    return df

#open all .txt files in a directory in xarray
sw_df = open_files('/run/media/sachin/0fa21ddb-f70c-4238-9cf4-705e0360f1c1/NICT DUMP/sw2020_2024/')
#sw_df = open_files('/run/media/sachin/0fa21ddb-f70c-4238-9cf4-705e0360f1c1/Test_SW/')


sw_df

<>:9: SyntaxWarning: invalid escape sequence '\s'
<>:9: SyntaxWarning: invalid escape sequence '\s'
/tmp/ipykernel_30108/1100088584.py:9: SyntaxWarning: invalid escape sequence '\s'
  data = pd.read_csv(directory+file, sep='\s+', header=None)


,dt,np,vsw,By,Bz
0,2020-08-01 00:00:00,6.65,342.7144,1.3,1.89
1,2020-08-01 00:01:00,6.65,342.7144,1.3,1.89
2,2020-08-01 00:02:00,6.65,342.7144,1.3,1.89
3,2020-08-01 00:03:00,6.65,342.7144,1.3,1.89
4,2020-08-01 00:04:00,6.65,342.7144,1.3,1.89
...,...,...,...,...,...
1924366,2024-07-31 23:54:00,0.52,452.4660,-8.5,-1.94
1924367,2024-07-31 23:55:00,0.52,452.4660,-8.5,-3.12
1924368,2024-07-31 23:56:00,0.52,452.4660,-7.3,-4.68
1924369,2024-07-31 23:57:00,0.50,461.2972,-6.8,-4.90


In [4]:
def calculate_tilt_angle(row):
    t0 = np.datetime64('1970-01-01T00:00:00')
    t1 = row['dt']
    tdiff = (t1 - t0).total_seconds()
    tilt_angle = geopack.recalc(tdiff) # Calculate dipole tilt angle
    
    return tilt_angle

sw_df['tilt_angle'] = sw_df.apply(calculate_tilt_angle, axis=1)


In [5]:
#resaample to 2 mins and interpolate over nan
sw_df = sw_df.resample('2min', on='dt').mean()
sw_df = sw_df.interpolate()
sw_df

,np,vsw,By,Bz,tilt_angle
dt,,,,,
2020-08-01 00:00:00,6.65,342.7144,1.30,1.890,0.264713
2020-08-01 00:02:00,6.65,342.7144,1.30,1.890,0.263360
2020-08-01 00:04:00,6.65,342.7144,1.30,1.890,0.262011
2020-08-01 00:06:00,6.65,342.7144,1.30,1.890,0.260666
2020-08-01 00:08:00,6.65,342.7144,1.30,1.890,0.259325
...,...,...,...,...,...
2024-07-31 23:50:00,0.50,446.5076,-9.70,-0.010,0.272255
2024-07-31 23:52:00,0.52,452.4660,-9.10,-1.665,0.270912
2024-07-31 23:54:00,0.52,452.4660,-8.50,-2.530,0.269573


In [6]:
export_path = f'/home/sachin/Documents/NIPR/Research/Data/OMNI/1-min/smrai3/' #server
df_name = f'rtsw_2min_2020-2024'
sw_df.to_csv(export_path + df_name + '.csv', index=True)